In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, GlobalMaxPooling1D, \
Concatenate, BatchNormalization, Conv1D, ReLU, Input, GRU
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime



In [ ]:
data = pd.read_csv('datasets/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

In [ ]:
def fill_na_with_adjacent_mean(column):
    missing_indices = data.index[data[column].isnull()]

    for idx in missing_indices:
        if idx != 0 and idx != data.index[-1]:
            mean_value = (data.at[idx-1, column] + data.at[idx+1, column]) / 2
            data.at[idx, column] = mean_value

In [ ]:
data = data.dropna()

In [ ]:
data_cleaned = data[['Timestamp','Weighted_Price']]
data_cleaned

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(data_cleaned['Weighted_Price'].values.reshape(-1,1))
data_cleaned['Weighted_Price'] = scaler.transform(data_cleaned['Weighted_Price'].values.reshape(-1,1))
data_cleaned

In [ ]:
generator = TimeseriesGenerator(data_cleaned['Weighted_Price'].values, data_cleaned['Weighted_Price'].values, length=10, batch_size=512)

In [ ]:
model = Sequential([
    LSTM(64,return_sequences=True,input_shape=(10,1)),
    LSTM(32,return_sequences=True),
    LSTM(16,return_sequences=False),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
history = model.fit(generator, epochs=2)

In [ ]:
model = tf.keras.models.load_model('models/btc.h5')

In [ ]:
date_string = '2023-09-01'
# Convert the date string to a datetime object
date_object = datetime.strptime(date_string, '%Y-%m-%d')
# Convert the datetime object to a Unix timestamp
timestamp = int(datetime.timestamp(date_object))
generator = TimeseriesGenerator([timestamp,timestamp],[timestamp,timestamp], length=1, batch_size=128)
print(timestamp)

val = model.predict(generator)
scaler.inverse_transform(val)